In [1]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor

from itertools import combinations
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
plt.style.use('seaborn-whitegrid')

C:\Users\sesa490527\AppData\Local\Continuum\anaconda2\envs\py36\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def read_data(filename, encoding='utf-8', sep=',',  parse_dates= ['timestamp'], index_col = ['timestamp']):
    data =  pd.read_csv(filename, encoding=encoding, sep=sep, parse_dates=parse_dates)
    if index_col != None: data.set_index(index_col, inplace=True)
    return data

def col_names(fr, to = 0, prefix = 'f_'):
    if to == 0: to = fr
    return ['{}{}'.format(prefix,s) for s in range(fr,to + 1)]

def meta_frame():
    col_dict = {
        "perc_mass" : col_names(0,7),
        "perc_vol" : col_names(41,42),
        "perc_ppm" : col_names(46),
        "abs_celsius" : col_names(20,21) + col_names(25) + col_names(28,31),
        "abs_hertz" : col_names(36,39),
        "abs_kg_h" : col_names(8,15) + col_names(17,19) + col_names(22,24) + col_names(26,27) 
                    + col_names(32,35) + col_names(43,45) + col_names(47),
        "abs_kpa" : col_names(16),
        "abs_mps" : col_names(40)
    }
    
    col_type_dict = {
        "abs" : [item for sublist in [col_dict[col] for col in col_dict if "abs" in col] for item in sublist],
        "perc" : [item for sublist in [col_dict[col] for col in col_dict if "perc" in col] for item in sublist]
    }
    return col_dict, col_type_dict

def resample_meta_frame(feature_names):
    features = {}
    level1, level2 = meta_frame()

    for key in level1.keys():
        for f in level1[key]:
            if f not in features:
                features_dict = { 'level1' : key }
                features[f] = features_dict
                
    for key in level2.keys():
        for f in level2[key]:
            features[f]['level2'] = key
    return pd.DataFrame(features).T

In [3]:
def split_train_test(data, labels, verbose=True):
    X_train = data[:"2017-10-31 23:00:00"].copy()
    y_train = labels[:"2017-10-31 23:00:00"].copy()
    X_val =  data["2017-11-01 00:00:00":"2017-12-31 23:00:00"].copy()
    y_val = labels["2017-11-01 00:00:00":"2017-12-31 23:00:00"].copy()
    X_test = data['2018-01-01 00:00:00':]
    if verbose:
        print('Train size:',X_train.shape, y_train.shape)
        print('Validation size:',X_val.shape, y_val.shape)
        print('Test size:', X_test.shape)
    return (X_train, y_train), (X_val, y_val), X_test

In [4]:
def preprocess(data):
    data_copy = data.copy()
    data_copy.loc['2017-06-16 14:00:00', ['f_41', 'f_42']] = np.nan #remove single outlier spike
    data_copy.loc['2017-02-09 14:00:00', ['f_41', 'f_42']] = np.nan #remove single outlier spike
    #data_copy.loc['2017-09-02':'2017-09-18 12:00:00', :] = np.nan # setting standstill to nan (for further drop)
    data_copy = data_copy.ffill().fillna(data_copy.loc[:"2017-12-31 23:00:00",:].mean())
    return data_copy

def drop_standstill(data, fr='2017-09-02 12:00:00', to='2017-09-20 12:00:00'):
    data_copy = data.copy()
    data_copy.drop(data_copy.loc[fr:to,:].index, inplace=True)
    return data_copy

def scale(data):
    scaler = StandardScaler()
    return scaler.fit_transform(data)

In [5]:
# Loading data
tags = read_data('tags.csv', index_col='feature', parse_dates=None)#pd.read_csv('tags.csv', encoding='utf-8', sep=',')
data = read_data('sensors.csv', parse_dates=['timestamp'], index_col=['timestamp'])
labels = read_data('coke_target.csv', parse_dates=['timestamp'], index_col=['timestamp'])

level1, level2 = meta_frame()
meta = resample_meta_frame(data.columns)
meta.head(1)

,level1,level2
f_0,perc_mass,perc


In [6]:
# Preprocessing data
data_prep = preprocess(data)
data_final = drop_standstill(data_prep)
labels_prep = labels.copy()
labels_final = drop_standstill(labels)

print('Original: {} // NaNs: {}'.format(data.shape, data.isna().sum().sum()))
print('Preprocessed: {} // NaNs: {}'.format(data_prep.shape, data_prep.isna().sum().sum()))
print('Preprocessed + drop standstill: {} // NaNs: {}'.format(data_final.shape, data_final.isna().sum().sum()))

Original: (13272, 48) // NaNs: 104053
Preprocessed: (13272, 48) // NaNs: 0
Preprocessed + drop standstill: (12839, 48) // NaNs: 0


# Modelling

In [7]:
def rmse(y_target, y_pred):
    return np.sqrt(mean_squared_error(y_target, y_pred))

def grid_search_cv(model, data, labels, params, cv, name='model'):
    (X_train, y_train), (X_val, y_val), X_test = split_train_test(data, labels, verbose=False)
    grid_search = GridSearchCV(model,
                               param_grid=params,
                               cv=cv,
                               scoring="neg_mean_squared_error",
                               n_jobs=1,
                               verbose=1, 
                               return_train_score=True)
    grid_search.fit(X_train, y_train['target'])
    print("Model best cv parameters:", grid_search.best_params_)
    models[name] = {"model": grid_search, "X_test" : X_test}
    
    train_predict = grid_search.predict(X_train)
    val_predict = grid_search.predict(X_val)
    print("RMSE train data", rmse(y_train['target'], train_predict))
    print("RMSE validation data:", rmse(y_val['target'], val_predict))
    
def submit(model, X_test, name, prefix='result_'):
    y_pred = model.predict(X_test)
    submission = pd.DataFrame(y_pred, index=X_test.index, columns=["target"])
    submission.to_csv("{}{}.csv".format(prefix,name))   
    
def submit_dict(models, model_name, X_test, name, prefix='result_'):
    y_pred = models[model_name]['model'].predict(X_test)
    submission = pd.DataFrame(y_pred, index=X_test.index, columns=["target"])
    submission.to_csv("{}{}.csv".format(prefix,name))

In [8]:
models={}
cv = TimeSeriesSplit(3)

In [9]:
model_gb = GradientBoostingRegressor(random_state=3)

### Feature engineering

In [43]:
def shift_features(data, features, shift=1, prefix='_shift', postfix=''):
    temp_data = pd.DataFrame()
    new_feature_names = list(map(lambda x: x+'{}{}{}'.format(prefix, shift, postfix), features))
    temp_data[new_feature_names] = data[features].shift(shift).fillna(0)
    return temp_data

def rolling_features(data, features, rolling=3, func='mean',prefix='_roll', postfix=''):
    temp_data = pd.DataFrame()
    new_feature_names = list(map(lambda x: x+'{}{}{}_{}'.format(prefix, rolling, postfix, func), features))
    temp_data[new_feature_names] = data[features].rolling(rolling).agg(func).fillna(0) 
    return temp_data

def ewm_features(data, features, func='mean', com=None, span=None, halflife=None, alpha=None, prefix='_ewm', postfix='', ):
    temp_data = pd.DataFrame()
    new_feature_names = list(map(lambda x: x+'{}{}_{}'.format(prefix, postfix, func), features))
    temp_data[new_feature_names] = data[features].ewm(com, span, halflife, alpha).agg(func).fillna(0) 
    return temp_data

def product_features(data, features, length=2):
    temp_data = pd.DataFrame()
    combs = combinations(features, length)
    for comb in combs:
        joiner = "".join
        features = list(map(joiner, comb))
        feat_name = "_".join(features)
        temp_data[feat_name] = data[features[0]]
        for f in features[1:]:
            temp_data[feat_name] = temp_data[feat_name] * data[f]
    return temp_data
    
def product_features_from_lists(data, list1, list2):
    temp_data = pd.DataFrame()
    combs =combinations(list1, list2)
    for comb in combs:
        joiner = "".join
        features = list(map(joiner, comb))
        feat_name = "_".join(features)
        temp_data[feat_name] = data[features[0]]
        for f in features[1:]:
            temp_data.loc[:, feat_name] = temp_data.loc[:, feat_name] * data[f] * 1.0
    return temp_data

def combinations(list1, list2):
    return [(x,y) for x in list1 for y in list2]

def concat(*dfs, axis=1):
    output = []
    for i, df in enumerate(dfs):
        output.append(df)
    return pd.concat(output, sort=False, axis=axis)

### Baseline

In [ ]:
model_gb_params = {
    'learning_rate' : [0.01],
    'n_estimators' : [100, 200, 300],
    'subsamples' : [1.0, 0.8]
}

### Гипотеза 1: f8=f9=f10 (корреляция 1) f11=f12=f13 (корреляция 1)

In [79]:
gb_params = {
    'learning_rate' : [0.01],
    'n_estimators' : [100,200,300],
    'max_depth' : [3],
    'subsample' : [1.0, 0.8]
}
hyp1_cols = ['f_8','f_9', 'f_10', 'f_44']
#hyp2_cols = ['f_11','f_12', 'f_13', 'f_43']
#hyp3_cols = ['f_20','f_28', 'f_30', 'f_31']
hyp4_cols = ['f_21','f_25', 'f_29', 'f_46'] # f_46 has -0.95 corr
hyp_cols_total = [hyp1_cols, hyp4_cols] #, hyp2_cols, hyp3_cols, hyp4_cols

data_hyp = data_prep.copy()
for hyp_cols in hyp_cols_total:
    data_hyp['_'.join(hyp_cols)+'_sum'] = data_hyp[hyp_cols].sum(axis=1).values
    data_hyp.drop(hyp_cols, axis=1, inplace=True)
data_hyp.head(1)

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_11,f_12,...,f_38,f_39,f_40,f_41,f_42,f_43,f_45,f_47,f_8_f_9_f_10_f_44_sum,f_21_f_25_f_29_f_46_sum
timestamp,,,,,,,,,,,,,,,,,,,,,
2016-10-24 16:00:00,99.087045,0.250268,26.505533,-0.005383,6.25191,0.022304,1.466704,11.173872,10388.134152,10301.147575,...,48.786063,48.719167,0.694998,0.005186,0.00409,10518.438958,10271.91295,64589.460605,947003.488612,1921.438795


In [25]:
grid_search_cv(model_gb, data_hyp, labels, gb_params, cv, name='model_hypl_14')

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:   46.0s finished


Model best cv parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
RMSE train data 0.41963482071157443
RMSE validation data: 0.2695977842593364


### Гипотеза 2 (новые фичи улучшают качество)

In [74]:
data_feat = data_hyp.copy()
#prod_features_1 = product_features_from_lists(data_prep_hyp_calendar, level2['abs'], level2['perc'])
shift_features_1 = shift_features(data_feat, data_feat.columns, shift=1)
roll_features_3 = rolling_features(data_feat, data_feat.columns, func='mean', rolling=3)


ewm_features_05 = ewm_features(data_feat, data_feat.columns, alpha=0.5)
#ewm_features_05 = ewm_features(data_prep_hyp_calendar, data_prep_hyp_calendar.columns, alpha=0.5)
data_feat = concat(data_feat, shift_features_1, 
                   roll_features_3)
#                   ewm_features_01, ewm_features_05)
data_feat.head(1)

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_11,f_12,...,f_38_roll3_mean,f_39_roll3_mean,f_40_roll3_mean,f_41_roll3_mean,f_42_roll3_mean,f_43_roll3_mean,f_45_roll3_mean,f_47_roll3_mean,f_8_f_9_f_10_f_44_sum_roll3_mean,f_21_f_25_f_29_f_46_sum_roll3_mean
timestamp,,,,,,,,,,,,,,,,,,,,,
2016-10-24 16:00:00,99.087045,0.250268,26.505533,-0.005383,6.25191,0.022304,1.466704,11.173872,10388.134152,10301.147575,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
# на обычных данных
grid_search_cv(model_gb, data_feat, labels, gb_params, cv, name='model1')

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  9.2min finished


Model best cv parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 300, 'subsample': 1.0}
RMSE train data 0.289692032973089
RMSE validation data: 0.29323755050111777


### Гипотеза 3 - парные признаки дают прирост

In [80]:
prod_feat = product_features_from_lists(data_feat, data_feat.columns[0:15], data_feat.columns[15:30])
data_feat_prod = pd.concat([data_feat, prod_feat], axis=1)
data_feat_prod.head(1)

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_11,f_12,...,f_17_f_24,f_17_f_26,f_17_f_27,f_17_f_28,f_17_f_30,f_17_f_31,f_17_f_32,f_17_f_33,f_17_f_34,f_17_f_35
timestamp,,,,,,,,,,,,,,,,,,,,,
2016-10-24 16:00:00,99.087045,0.250268,26.505533,-0.005383,6.25191,0.022304,1.466704,11.173872,10388.134152,10301.147575,...,5382.137124,1363.918103,4314.223097,25862.461986,25480.464796,26337.058367,4913.158626,388.731396,358.051983,389.704637


In [81]:
# на обычных данных
grid_search_cv(model_gb, data_feat_prod, labels, gb_params, cv, name='model1')

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:  7.8min finished


Model best cv parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
RMSE train data 0.42219355061313635
RMSE validation data: 0.27931579926542655
